
 # Import libraries and include colab utils


In [ ]:
use_gdrive = True
gdrive_path = "FIND-master"

force_rebuild = True

In [ ]:
!pip install tensorflow==1.15.0
!pip install keras==2.2.4

     |████████████████████████████████| 412.3MB 26kB/s 
     |████████████████████████████████| 3.8MB 35.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 512kB 27.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=faa505d7f905c6de9159b5fd42cfb1a46b40ab68514435966b4c54164f0b271c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

     |████████████████████████████████| 317kB 5.2MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
""" Colab gdrive utils """
from google.colab import drive
from pathlib import Path
import os

def mount_drive():
    if use_gdrive:
        drive.mount('/content/drive')
        os.chdir('/content/drive/MyDrive')


        #Set working dir as given gdrive root
        os.chdir('/content/drive/MyDrive/'+gdrive_path)
        print(f'Working directory set to: \"{os.getcwd()}\"')

def unmount_drive():
    if use_gdrive:
        drive.flush_and_unmount()
        os.chdir('/content')

mount_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working directory set to: "/content/drive/MyDrive/FIND-master"


In [ ]:
# Notebook setup
import pickle
import os
import datetime
import random
import numpy as np
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14, 7]
os.environ['PYTHONHASHSEED'] = '0'

# Set random seed to create reproducable results
the_seed = 1234
np.random.seed(the_seed)
random.seed(the_seed)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from tensorflow.keras import backend as K
tf.set_random_seed(the_seed)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

1.15.0


In [ ]:
import find

Using TensorFlow backend.


## Settings
using YELP setting as an example to show how to load and run a pre-trained model for FIND.

In [ ]:
EMBEDDING_DIM = 300
EMBEDDING_PATH = f"./embeds/glove/glove.6B.{EMBEDDING_DIM}d.txt" # Path to your glove embeddings

In [ ]:
DATA_PATH = './preprocessed_data/'
MAIN_DATASET = 'YelpSmall500'
SECOND_DATASET = None
THIRD_DATASET = None
GENDER_BIAS = False

In [ ]:
MAIN_DATASET

'YelpSmall500'

In [ ]:
MODEL_FILE_PATH = './trained_models/tmp/YelpSmall500_CNN_20200515025212.h5'
MODEL_ARCH = 'CNN'
MAXLEN = 150
FILTERS = [(10, 2), (10, 3), (10, 4)] # Ten filters of each window size [2,3,4]
BATCH_SIZE = 128

## Prepare embeddings and dataset

In [ ]:
# 0. Load GloVe embeddings
embedding_matrix, vocab_size, index2word, word2index = find.get_embedding_matrix(EMBEDDING_PATH, EMBEDDING_DIM, pad_initialisation = "zeros")

Loading Glove Model


400000it [00:51, 7708.99it/s]


Done. 400000  words loaded!


In [ ]:
# 1. Load datasets and prepare inputs
# 1.1 Main dataset
data_1 = pickle.load(open(DATA_PATH + f'all_data_{MAIN_DATASET}.pickle', 'rb'))
class_names = data_1['class_names']
X_train_1, X_validate_1, X_test_1 = find.get_data_matrix(data_1['text_train'], word2index, MAXLEN), \
                                    find.get_data_matrix(data_1['text_validate'], word2index, MAXLEN), \
                                    find.get_data_matrix(data_1['text_test'], word2index, MAXLEN)
y_test_1 = data_1['y_test']
#gender_test_1 = data_1['gender_test'] if GENDER_BIAS else None

# 1.2 Second dataset
if SECOND_DATASET is not None:
    data_2 = pickle.load(open(DATA_PATH + f'all_data_{SECOND_DATASET}.pickle', 'rb'))
    X_test_2, y_test_2 = find.get_data_matrix(data_2['text_test'], word2index, MAXLEN), data_2['y_test']
    #gender_test_2 = data_2['gender_test'] if GENDER_BIAS else None
else:
    X_test_2, y_test_2 = None, None
    #gender_test_2 =  None

# 1.3 Third dataset
if THIRD_DATASET is not None:
    data_3 = pickle.load(open(DATA_PATH + f'all_data_{THIRD_DATASET}.pickle', 'rb'))
    X_test_3, y_test_3 = find.get_data_matrix(data_3['text_test'], word2index, MAXLEN), data_3['y_test']
    #gender_test_3 = data_3['gender_test'] if GENDER_BIAS else None
else:
    X_test_3, y_test_3 =  None, None 
    #gender_test_2  = None

In [ ]:
train_dict = dict([(data_1['train_text'], data_1['train_label'])])

NameError: ignored

## Create, load, and run the model

In [ ]:
# 3. Create a model
if MODEL_ARCH == 'CNN':
    model = find.get_CNN_model(vocab_size, EMBEDDING_DIM, embedding_matrix, MAXLEN, class_names, FILTERS)
else:
    assert False, f"Unsupported model architecture: {MODEL_ARCH}"

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 300)     120000600   input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 149, 10)      6010        embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 148, 10)      9010        embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (

In [ ]:
# 4. Load pre-trained model weights
model.load_weights(MODEL_FILE_PATH)

In [ ]:
# 5. Evaluate the model for YELP dataset(no gender bias)
if not GENDER_BIAS:
    find.evaluate_all(model, class_names, BATCH_SIZE, X_test_1, y_test_1, X_test_2, y_test_2, X_test_3, y_test_3, result_path = None, model_name = 'original')
else:
    find.evaluate_all_gender(model, class_names, BATCH_SIZE, X_test_1, y_test_1, gender_test_1, X_test_2, y_test_2, gender_test_2, result_path = None, model_name = 'original')

Evaluate with the original test set:
{'per_class': {0: {'all_positive': 25219,
                   'all_true': 19000,
                   'class_f1': 0.7082023564531085,
                   'class_name': 'Negative',
                   'class_precision': 0.620881081724097,
                   'class_recall': 0.8241052631578948,
                   'true_positive': 15658},
               1: {'all_positive': 12781,
                   'all_true': 19000,
                   'class_f1': 0.5940027060193197,
                   'class_name': 'Positive',
                   'class_precision': 0.7385181128237227,
                   'class_recall': 0.4967894736842105,
                   'true_positive': 9439}},
 'total': {'accuracy': 0.6604473684210527,
           'macro_f1': 0.6699351964037961,
           'macro_precision': 0.6796995972739098,
           'macro_recall': 0.6604473684210527,
           'micro_f1': 0.6604473684210527,
           'micro_precision': 0.6604473684210527,
           'micro_reca

In [ ]:
# 2. Create the result directory
if not os.path.exists(MODEL_FILE_PATH):
    os.makedirs(MODEL_FILE_PATH)
result_folder = MAIN_DATASET + '_' + MODEL_ARCH + '_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + '/'
result_path = MODEL_FILE_PATH + result_folder
os.mkdir(result_path)

In [ ]:
settings = {
    'model_arch': MODEL_ARCH,
    'filters': FILTERS,
    'maxlen': MAXLEN,
    'result_path': result_path,
    'index2word': index2word,
    'embedding_dim': EMBEDDING_DIM,
    'batch_size': BATCH_SIZE
}
all_wordclouds = find.generate_wordclouds(model, X_train_1, settings, max_examples = 2000)

  0%|          | 0/30 [00:00<?, ?it/s]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedded_text_input (InputLayer (None, 150, 300)     0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 149, 10)      6010        embedded_text_input[0][0]        
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 148, 10)      9010        embedded_text_input[0][0]        
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 147, 10)      12010       embedded_text_input[0][0]        
__________________________________________________________________________________________________
global_max

100%|██████████| 30/30 [00:20<00:00,  1.48it/s]


In [ ]:
is_feature_enabled = [True for i in range(find.num_all_filters(FILTERS))]

In [ ]:
# UI components from ipywidgets
import ipywidgets as wgt

def update_screen(feature_idx):
    show_action_panel(feature_idx)
    wordcloud = all_wordclouds[feature_idx]
    f, ax = plt.subplots()
    plt.rcParams['figure.figsize'] = [14, 7]
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")
    
    W = model.layers[-1].get_weights()[0] # For the final layer
    weight_plot = find.visualize_weights(W, feature_idx, class_names, show = False)
    plt.show()

def update_action(action):
    global feature_radio_button, is_feature_enabled
    feature_idx = feature_radio_button.value
    if action == 'enabled':
        print('enable')
        is_feature_enabled[feature_idx] = True
    elif action == 'disabled':
        print('disable')
        is_feature_enabled[feature_idx] = False
    else:
        assert False
    
def show_action_panel(feature_idx):
    global action_radio_button
    action_radio_button.description = f'Current status of feature {feature_idx}:'
    action_radio_button.value = 'enabled' if is_feature_enabled[feature_idx] else 'disabled'
    
feature_radio_button = wgt.RadioButtons(options=list(range(30)), value=0, description='Feature:', disabled=False)
action_radio_button = wgt.RadioButtons(options=['enabled', 'disabled'],
    value = 'enabled' if is_feature_enabled[feature_radio_button.value] else 'disabled',
    description = f'Current status of feature {feature_radio_button.value}:',
    style = {'description_width': 'initial'},
    disabled = False
)

wgt.interactive_output(update_action, {'action':action_radio_button})
out = wgt.interactive_output(update_screen, {'feature_idx':feature_radio_button})

In [ ]:
# 7. Get input from a human 
# Please investigate word clouds of these features and disable some irrelevant features using the radio-buttons under the bar plot.
# In particular, to reduce the model's bias, we should disable the features which detect opposite information rather than occupation.
# Once you are happy, please then proceed to the next cell.
display(wgt.HBox([feature_radio_button, wgt.VBox([out, action_radio_button])]))

In [ ]:
print(f"Total: {len(is_feature_enabled)} features \nEnabled: {sum(is_feature_enabled)} features \nDisabled: {len(is_feature_enabled)-sum(is_feature_enabled)} features")
print(f"Disabled features: {[i for i,s in enumerate(is_feature_enabled) if not s]}")

Total: 30 features 
Enabled: 25 features 
Disabled: 5 features
Disabled features: [0, 1, 2, 3, 8]


In [ ]:
# 8. Create an improved model
# 8.1 Copy the existing CNN features
model_improved = find.get_CNN_model(vocab_size, EMBEDDING_DIM, embedding_matrix, MAXLEN, class_names, 
                                    FILTERS, trainable_filters = False)
model_improved.set_weights(model.get_weights()) 

# 8.2 Apply human decisions to disable irrelevant features
for idx, enable in enumerate(is_feature_enabled):
    if not enable:
        model_improved.layers[-1].disable_mask(idx)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 150, 300)     120000600   input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 149, 10)      6010        embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 148, 10)      9010        embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_12 

In [ ]:
# 9. Fine-tuning the improved model
history = find.model_train(model_improved, result_path + f'trained_{MODEL_ARCH}_improved.h5', X_train_1, data_1['y_train'], X_validate_1, data_1['y_validate'], BATCH_SIZE, epochs = 300)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 500 samples, validate on 100 samples
Epoch 1/300
 - 0s - loss: 0.3935 - acc: 0.9460 - val_loss: 0.6158 - val_acc: 0.6700

Epoch 00001: val_loss improved from inf to 0.61581, saving model to ./trained_models/tmp/YelpSmall500_CNN_20200515025212.h5YelpSmall500_CNN_20210423144442/trained_CNN_improved.h5
Epoch 2/300
 - 0s - loss: 0.3716 - acc: 0.9720 - val_loss: 0.6190 - val_acc: 0.6500

Epoch 00002: val_loss did not improve from 0.61581
Epoch 3/300
 - 0s - loss: 0.3589 - acc: 0.9840 - val_loss: 0.6281 - val_acc: 0.6700

Epoch 00003: val_loss did not improve from 0.61581
Epoch 4/300
 - 0s - loss: 0.3534 - acc: 0.9620 - val_loss: 0.6388 - val_acc: 0.6500

Epoch 00004: val_loss did not improve from 0.61581


In [ ]:
if not GENDER_BIAS:
    find.evaluate_all(model_improved, class_names, BATCH_SIZE, X_test_1, y_test_1, X_test_2, y_test_2, X_test_3, y_test_3, result_path = result_path, model_name = 'debugged')
else:
    find.evaluate_all_gender(model_improved, class_names, BATCH_SIZE, X_test_1, y_test_1, gender_test_1, X_test_2, y_test_2, gender_test_2, result_path = result_path, model_name = 'debugged')

Evaluate with the original test set:
{'per_class': {0: {'all_positive': 19484,
                   'all_true': 19000,
                   'class_f1': 0.6786196861033156,
                   'class_name': 'Negative',
                   'class_precision': 0.6701909258879081,
                   'class_recall': 0.6872631578947368,
                   'true_positive': 13058},
               1: {'all_positive': 18516,
                   'all_true': 19000,
                   'class_f1': 0.6703273270071436,
                   'class_name': 'Positive',
                   'class_precision': 0.6790883560164183,
                   'class_recall': 0.6617894736842105,
                   'true_positive': 12574}},
 'total': {'accuracy': 0.6745263157894736,
           'macro_f1': 0.6745829736113622,
           'macro_precision': 0.6746396409521631,
           'macro_recall': 0.6745263157894736,
           'micro_f1': 0.6745263157894736,
           'micro_precision': 0.6745263157894736,
           'micro_re